In [1]:
# import logging

# from autogen_core import EVENT_LOGGER_NAME

# logging.basicConfig(level=logging.WARNING)
# logger = logging.getLogger(EVENT_LOGGER_NAME)
# logger.addHandler(logging.StreamHandler())
# logger.setLevel(logging.INFO)

In [2]:
import asyncio

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo, UserMessage

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

model_client = OpenAIChatCompletionClient(
        model="Devstral",
        base_url='http://ai-developer.behavior-analysis.k8s.antifraud-xc:8080/v1',
        api_key="mykey",
        model_info=ModelInfo(vision=False, function_calling=False, json_output=False, family="mistral", structured_output=True),
    )

In [3]:
assistant_system_message = ('You are an expert Python developer with deep knowledge of software testing, including unit tests, integration tests, and best practices for test-driven development (TDD). Your task is to analyze Python code and generate high-quality, maintainable, and efficient test cases. Instructions:  1. Understand the Code:    - Carefully review the provided Python code, including functions, classes, and dependencies.    - Identify edge cases, expected behavior, and potential failure points.  2. Choose a Testing Framework:     - Default to `pytest` if no framework is specified.     - Ensure compatibility with the code’s dependencies (e.g., `mock` for external services).  3. Generate Tests:  - Write tests that cover:       - Happy Path (normal operation).       - Edge Cases (empty inputs, invalid types, boundary conditions).       - Error Handling (exceptions, invalid states).     - Include descriptive docstrings or comments explaining each test’s purpose.     - Use fixtures (`pytest`) or `setUp/tearDown` (`unittest`) where needed.  4. Output Format:     - Return the test code in a complete, executable format.     - If the original code has bugs, note them and write tests to catch them.  When done, say TERMINATE.')

In [4]:
primary_agent = AssistantAgent(
    "test_writer",
    model_client=model_client,
    system_message=assistant_system_message,
)

In [5]:
critic_system_message = "You are a task verification assistant who is working with a test writer agent to solve tasks. At each point, check if the task has been completed as requested by the user. You MUST run and validate all the tests provided by test_writer as completed. If the test_writer responds and the task has not yet been completed, respond with what is left to do and then say 'keep going' (provide constructive feedback). If and only when the task has been completed, summarize and present a final answer that directly addresses the user task in detail and then respond with APPROVE."
# critic_system_message = "Provide constructive feedback. Respond with 'APPROVE' to obly after ALL your feedbacks are addressed by test_writer."

In [6]:
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message=critic_system_message
)

In [7]:
text_termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [9]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a function in Python and several tests for it."))

---------- TextMessage (user) ----------
Write a function in Python and several tests for it.
---------- TextMessage (test_writer) ----------
Sure, let's start by writing a simple Python function and then I'll create tests for it.

### Function: `calculate_factorial`

This function will calculate the factorial of a given number. The factorial of a non-negative integer `n` is the product of all positive integers less than or equal to `n`.

```python
def calculate_factorial(n):
    """
    Calculate the factorial of a non-negative integer.

    Args:
    n (int): A non-negative integer.

    Returns:
    int: The factorial of the given number.

    Raises:
    ValueError: If the input is not a non-negative integer.
    """
    if not isinstance(n, int):
        raise ValueError("Input must be an integer.")
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")

    result = 1
    for i in range(1, n + 1):
        result *= i
    return result
```

### Tests for `

TaskResult(messages=[TextMessage(id='f3992772-5ec1-43c6-8476-cdafa9b232b5', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 13, 17, 34, 47, 50571, tzinfo=datetime.timezone.utc), content='Write a function in Python and several tests for it.', type='TextMessage'), TextMessage(id='76e7ec04-30d8-4f0c-94c5-6906342cb952', source='test_writer', models_usage=RequestUsage(prompt_tokens=284, completion_tokens=734), metadata={}, created_at=datetime.datetime(2025, 8, 13, 17, 35, 7, 437770, tzinfo=datetime.timezone.utc), content='Sure, let\'s start by writing a simple Python function and then I\'ll create tests for it.\n\n### Function: `calculate_factorial`\n\nThis function will calculate the factorial of a given number. The factorial of a non-negative integer `n` is the product of all positive integers less than or equal to `n`.\n\n```python\ndef calculate_factorial(n):\n    """\n    Calculate the factorial of a non-negative integer.\n\n    Args:\n    n (int): 

In [7]:
# result = await team.run(task="Write a simple but useful function in Python and a test for it.")
# print(result.messages[1].content)

In [10]:
# pip install --upgrade autogen-agentchat

In [9]:
import asyncio
from autogen_agentchat.agents import CodeExecutorAgent, ApprovalRequest, ApprovalResponse
from autogen_agentchat.messages import TextMessage
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
from autogen_core import CancellationToken


def simple_approval_func(request: ApprovalRequest) -> ApprovalResponse:
    """Simple approval function that requests user input for code execution approval."""
    print("Code execution approval requested:")
    print("=" * 50)
    print(request.code)
    print("=" * 50)

    while True:
        user_input = input("Do you want to execute this code? (y/n): ").strip().lower()
        if user_input in ['y', 'yes']:
            return ApprovalResponse(approved=True, reason='Approved by user')
        elif user_input in ['n', 'no']:
            return ApprovalResponse(approved=False, reason='Denied by user')
        else:
            print("Please enter 'y' for yes or 'n' for no.")


async def run_code_executor_agent() -> None:
    # Create a code executor agent that uses a Docker container to execute code.
    code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
    await code_executor.start()
    code_executor_agent = CodeExecutorAgent(
        "code_executor",
        code_executor=code_executor,
        approval_func=simple_approval_func
    )

    # Run the agent with a given code snippet.
    task = TextMessage(
        content='''Here is some code
```python
print('Hello world')
```
''',
        source="user",
    )
    response = await code_executor_agent.on_messages([task], CancellationToken())
    print(response.chat_message)

    # Stop the code executor.
    await code_executor.stop()


await run_code_executor_agent()
# asyncio.run(run_code_executor_agent())

Code execution approval requested:
```python
print('Hello world')

```


Do you want to execute this code? (y/n):  y


id='76fcaa10-f6a1-44c3-9bd4-1a5dcd132e51' source='code_executor' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 12, 18, 58, 32, 703625, tzinfo=datetime.timezone.utc) content='Hello world\n' type='TextMessage'


In [1]:
# !pip install "autogen-ext[docker]"
# !pip install docker

In [ ]:
# CodeExecutorAgent
# sources (Sequence[str], optional) – Check only messages from the specified agents for the code to execute
# supported_languages

# The code executor only processes code that is properly formatted in markdown code blocks using triple backticks

In [13]:
# pip install "autogen-ext[file-surfer]"